# mzPeak Quickstart Tutorial

This notebook demonstrates the complete mzPeak workflow:
1. **Convert** mzML to mzPeak format
2. **Read** and explore the data
3. **Visualize** with mirror plots

⏱️ **Time to complete:** < 5 minutes

## Prerequisites

```bash
# Install required packages
pip install pyarrow pandas matplotlib numpy
```

## Step 1: Generate Demo Data

First, let's create a demo mzPeak file using the command-line tool.

In [ ]:
# Generate demo LC-MS data
!mzpeak demo demo_run.mzpeak

## Step 2: Read mzPeak Data with PyArrow

mzPeak files are Apache Parquet files that can be read by any Parquet-compatible tool.

In [ ]:
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read the mzPeak file (ZIP container or directory bundle)
# For .mzpeak container (ZIP), extract first or read from extracted directory
table = pq.read_table('demo_run.mzpeak/peaks/peaks.parquet')
df = table.to_pandas()

print(f"Loaded {len(df):,} peaks from {df['spectrum_id'].nunique():,} spectra")
df.head()

## Step 3: Explore the Data

Let's look at the dataset structure and statistics.

In [ ]:
# Dataset summary
print("Dataset Summary:")
print(f"  Total spectra: {df['spectrum_id'].nunique():,}")
print(f"  Total peaks: {len(df):,}")
print(f"  MS levels: {sorted(df['ms_level'].unique())}")
print(f"  m/z range: {df['mz'].min():.2f} - {df['mz'].max():.2f}")
print(f"  RT range: {df['retention_time'].min():.2f} - {df['retention_time'].max():.2f} sec")

# MS level distribution
ms_level_counts = df.groupby('ms_level').agg({
    'spectrum_id': 'nunique',
    'mz': 'count'
}).rename(columns={'spectrum_id': 'num_spectra', 'mz': 'num_peaks'})

print("\nMS Level Distribution:")
print(ms_level_counts)

## Step 4: Base Peak Chromatogram (BPC)

Visualize the chromatographic profile by plotting the maximum intensity per spectrum over time.

In [ ]:
# Calculate Base Peak Chromatogram
ms1_data = df[df['ms_level'] == 1].copy()
bpc = ms1_data.groupby('retention_time')['intensity'].max().reset_index()

# Plot
plt.figure(figsize=(12, 4))
plt.plot(bpc['retention_time'] / 60, bpc['intensity'], linewidth=0.8)
plt.xlabel('Retention Time (min)')
plt.ylabel('Intensity')
plt.title('Base Peak Chromatogram (BPC)')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Step 5: Extracted Ion Chromatogram (XIC)

Extract and plot the chromatogram for a specific m/z value.

In [ ]:
# Target m/z and tolerance (ppm)
target_mz = 500.0
ppm_tolerance = 10

# Calculate m/z window
mz_min = target_mz * (1 - ppm_tolerance * 1e-6)
mz_max = target_mz * (1 + ppm_tolerance * 1e-6)

# Filter peaks
xic_data = ms1_data[
    (ms1_data['mz'] >= mz_min) & 
    (ms1_data['mz'] <= mz_max)
].copy()

# Aggregate by retention time
xic = xic_data.groupby('retention_time')['intensity'].sum().reset_index()

# Plot
plt.figure(figsize=(12, 4))
plt.plot(xic['retention_time'] / 60, xic['intensity'], linewidth=1, color='#2E86AB')
plt.fill_between(xic['retention_time'] / 60, xic['intensity'], alpha=0.3, color='#2E86AB')
plt.xlabel('Retention Time (min)')
plt.ylabel('Intensity')
plt.title(f'Extracted Ion Chromatogram (XIC) - m/z {target_mz:.4f} ± {ppm_tolerance} ppm')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Found {len(xic_data)} peaks in XIC window")

## Step 6: MS1 Spectrum Visualization

Plot an individual MS1 spectrum.

In [ ]:
# Select a spectrum from the middle of the run
mid_rt = df['retention_time'].median()
spectrum_id = df[df['retention_time'] >= mid_rt]['spectrum_id'].iloc[0]

# Get spectrum data
spectrum = df[(df['spectrum_id'] == spectrum_id) & (df['ms_level'] == 1)].copy()
spectrum = spectrum.sort_values('mz')

# Plot
plt.figure(figsize=(12, 5))
plt.vlines(spectrum['mz'], 0, spectrum['intensity'], linewidth=0.8, color='#A23B72')
plt.xlabel('m/z')
plt.ylabel('Intensity')
plt.title(f'MS1 Spectrum (ID: {spectrum_id}, RT: {spectrum["retention_time"].iloc[0]/60:.2f} min)')
plt.xlim(spectrum['mz'].min() - 50, spectrum['mz'].max() + 50)
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"Spectrum contains {len(spectrum)} peaks")

## Step 7: Mirror Plot (MS2 Comparison)

Create a mirror plot to compare two MS2 spectra - a common visualization in proteomics.

In [ ]:
# Get MS2 spectra
ms2_data = df[df['ms_level'] == 2].copy()

if len(ms2_data) == 0:
    print("No MS2 spectra found in demo data.")
else:
    # Select two MS2 spectra with similar precursor m/z
    precursor_mzs = ms2_data.groupby('spectrum_id')['precursor_mz'].first()
    
    if len(precursor_mzs) >= 2:
        spec_ids = precursor_mzs.head(2).index
        spec1_id, spec2_id = spec_ids[0], spec_ids[1]
        
        # Get spectrum data
        spec1 = ms2_data[ms2_data['spectrum_id'] == spec1_id].sort_values('mz')
        spec2 = ms2_data[ms2_data['spectrum_id'] == spec2_id].sort_values('mz')
        
        # Normalize intensities
        spec1_norm = spec1.copy()
        spec2_norm = spec2.copy()
        spec1_norm['intensity'] = spec1_norm['intensity'] / spec1_norm['intensity'].max() * 100
        spec2_norm['intensity'] = spec2_norm['intensity'] / spec2_norm['intensity'].max() * 100
        
        # Create mirror plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Top spectrum (normal)
        ax.vlines(spec1_norm['mz'], 0, spec1_norm['intensity'], 
                 linewidth=1, color='#E63946', label=f'Spectrum {spec1_id}')
        
        # Bottom spectrum (inverted)
        ax.vlines(spec2_norm['mz'], 0, -spec2_norm['intensity'], 
                 linewidth=1, color='#457B9D', label=f'Spectrum {spec2_id}')
        
        # Styling
        ax.axhline(y=0, color='black', linewidth=0.8)
        ax.set_xlabel('m/z')
        ax.set_ylabel('Relative Intensity (%)')
        ax.set_title(
            f'MS/MS Mirror Plot\n'
            f'Precursor: {spec1["precursor_mz"].iloc[0]:.4f} Da (top) vs '
            f'{spec2["precursor_mz"].iloc[0]:.4f} Da (bottom)'
        )
        ax.legend()
        ax.grid(alpha=0.3, axis='x')
        
        # Set y-axis limits
        ax.set_ylim(-110, 110)
        
        plt.tight_layout()
        plt.show()
        
        print(f"Spectrum {spec1_id}: {len(spec1)} peaks, Precursor m/z: {spec1['precursor_mz'].iloc[0]:.4f}")
        print(f"Spectrum {spec2_id}: {len(spec2)} peaks, Precursor m/z: {spec2['precursor_mz'].iloc[0]:.4f}")
    else:
        print("Need at least 2 MS2 spectra for mirror plot.")

## Step 8: Query with DuckDB (Optional)

For SQL-based analysis, use DuckDB to query mzPeak files directly.

In [ ]:
try:
    import duckdb
    
    # Connect to in-memory database
    conn = duckdb.connect()
    
    # Query MS2 spectra with high-intensity fragments
    query = """
    SELECT 
        precursor_mz,
        COUNT(*) as num_fragments,
        MAX(intensity) as max_fragment_intensity,
        AVG(mz) as avg_fragment_mz
    FROM read_parquet('demo_run.mzpeak/peaks/peaks.parquet')
    WHERE ms_level = 2
    GROUP BY precursor_mz, spectrum_id
    HAVING num_fragments > 10
    ORDER BY max_fragment_intensity DESC
    LIMIT 10;
    """
    
    result = conn.execute(query).fetchdf()
    print("Top 10 MS2 Spectra by Fragment Intensity:")
    display(result)
    
    conn.close()
    
except ImportError:
    print("DuckDB not installed. Install with: pip install duckdb")
    print("See examples/duckdb_queries.sql for more query examples.")

## Step 9: Advanced - Ion Mobility Data (if available)

For datasets with ion mobility dimension (e.g., timsTOF):

In [ ]:
# Check if ion mobility data is present
has_im = df['ion_mobility'].notna().any()

if has_im:
    im_data = df[df['ion_mobility'].notna()].copy()
    
    print(f"Ion mobility data available: {len(im_data):,} peaks ({len(im_data)/len(df)*100:.1f}%)")
    print(f"IM range: {im_data['ion_mobility'].min():.2f} - {im_data['ion_mobility'].max():.2f} ms")
    
    # Create 2D histogram (m/z vs ion mobility)
    plt.figure(figsize=(12, 6))
    
    hist = plt.hist2d(
        im_data['mz'], 
        im_data['ion_mobility'],
        bins=[100, 50],
        cmap='viridis',
        norm=plt.matplotlib.colors.LogNorm()
    )
    
    plt.colorbar(hist[3], label='Peak Count (log scale)')
    plt.xlabel('m/z')
    plt.ylabel('Ion Mobility (ms)')
    plt.title('2D Heatmap: m/z vs Ion Mobility')
    plt.tight_layout()
    plt.show()
else:
    print("No ion mobility data in this dataset.")
    print("IM dimension is available for timsTOF, SYNAPT, and other IM-MS instruments.")

## Summary

This notebook demonstrated:

✅ **Conversion**: Generate mzPeak files from mzML using the CLI tool  
✅ **Reading**: Load mzPeak data with PyArrow/Pandas (zero dependencies)  
✅ **Exploration**: Calculate BPC, XIC, and dataset statistics  
✅ **Visualization**: Create publication-quality plots including mirror plots  
✅ **Integration**: Query with DuckDB for SQL-based analysis  
✅ **Advanced**: Support for ion mobility dimension (4D proteomics)  

### Key Advantages of mzPeak:

- **5-10x compression** over mzML
- **Universal compatibility** with Parquet tools (Python, R, DuckDB, Spark)
- **Blazing fast queries** via columnar storage and predicate pushdown
- **Single-file format** for easy distribution
- **Native ion mobility** support for 4D MS

### Next Steps:

- Convert your own mzML files: `mzpeak convert input.mzML output.mzpeak`
- Explore DuckDB queries: `examples/duckdb_queries.sql`
- Read the documentation: `README.md`
- Validate files: `mzpeak validate data.mzpeak`